In [ ]:
import pandas as pd
import numpy as np
import random
import locale


In [ ]:
# Criando as datas de 01/01/2024 até 30/04/2024
datas = pd.date_range(start='2024-01-01', end='2025-04-30', freq='D')

# Criando um DataFrame com as datas
df = pd.DataFrame({'data': datas})

# Define o locale para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')  # No Windows pode ser 'Portuguese_Brazil.1252'

# Gera o nome do dia da semana em português e pega os 3 primeiros caracteres
df['dia_semana'] = df['data'].dt.strftime('%A').str[:3].str.lower()

# Filtrando apenas dias úteis (segunda a sexta)
dias_uteis = df[df['dia_semana'].isin(['seg', 'ter', 'qua', 'qui', 'sex'])]

# Média de vendas nos dias úteis
amostra_vendas = np.random.randint(29, 58, size=len(dias_uteis))
media_uteis = int(np.mean(amostra_vendas))

# Função para gerar valores aleatórios de produto, oferta, valor e promo
def gera_valores_aleatorios():
    # Nome do produto
    letras = ['A', 'B', 'C', 'D']
    produto = random.choice(letras)

    # Gerando valores aleatórios 'S' ou 'N' para a coluna 'oferta'
    oferta = random.choice(['S', 'N'])

    # Mapeando os valores dos produtos
    valores = {'A': 129.90, 'B': 119.90, 'C': 109.90, 'D': 99.90}
    valor = valores[produto]

    # Lista de promoções possíveis
    promocoes = ['mensal', '30_P', '50_P']

    

    # Função para aplicar a lógica de promoção
    def atribui_promocao(oferta):
        if oferta == 'S':
            return random.choice(promocoes)
        else:
            return 'Sem_Oferta'

    # Atribuindo promoção
    promo = atribui_promocao(oferta)

    return produto, oferta, valor, promo

# Função para definir quantas vendas por dia
def num_vendas(dia):
    if dia == 'dom':
        return 0
    elif dia == 'sab':
        return max(1, media_uteis // 2)
    else:
        return random.randint(29, 57)
    

# Gerando vendas para cada linha, com valores aleatórios
df_vendas = df.loc[df.index.repeat(df['dia_semana'].apply(num_vendas))].reset_index(drop=True)

# Gerando valores aleatórios para cada linha de venda
df_vendas['produto'], df_vendas['oferta'], df_vendas['valor'], df_vendas['promo'] = zip(*[gera_valores_aleatorios() for _ in range(len(df_vendas))])

# Exibindo uma amostra das vendas geradas
print(df_vendas.head())

        data dia_semana produto oferta  valor       promo
0 2024-01-01        seg       C      S  109.9        50_P
1 2024-01-01        seg       D      N   99.9  Sem_Oferta
2 2024-01-01        seg       B      N  119.9  Sem_Oferta
3 2024-01-01        seg       A      N  129.9  Sem_Oferta
4 2024-01-01        seg       D      N   99.9  Sem_Oferta


In [38]:
# Exibindo as primeiras linhas do DataFrame
df_vendas

,data,dia_semana,produto,oferta,valor,promo,vigencia
0,2024-01-01,seg,C,S,109.9,50_P,3
1,2024-01-01,seg,D,N,99.9,Sem_Oferta,0
2,2024-01-01,seg,B,N,119.9,Sem_Oferta,0
3,2024-01-01,seg,A,N,129.9,Sem_Oferta,0
4,2024-01-01,seg,D,N,99.9,Sem_Oferta,0
...,...,...,...,...,...,...,...
17950,2025-04-30,qua,C,S,109.9,30_P,3
17951,2025-04-30,qua,D,N,99.9,Sem_Oferta,0
17952,2025-04-30,qua,C,N,109.9,Sem_Oferta,0
17953,2025-04-30,qua,B,S,119.9,50_P,3


In [41]:
# Criando a Vigencia

df_vendas['vigencia'] = df_vendas['promo'].apply(lambda x: 12 if x == 'mensal' else (0 if x == 'Sem_Oferta' else 3))

# Exibindo o DataFrame para verificar
print(df_vendas.head())


        data dia_semana produto oferta  valor       promo  vigencia  oferta_vl
0 2024-01-01        seg       C      S  109.9        50_P         3      54.95
1 2024-01-01        seg       D      N   99.9  Sem_Oferta         0      99.90
2 2024-01-01        seg       B      N  119.9  Sem_Oferta         0     119.90
3 2024-01-01        seg       A      N  129.9  Sem_Oferta         0     129.90
4 2024-01-01        seg       D      N   99.9  Sem_Oferta         0      99.90


In [42]:
# Criando valor da oferta
df_vendas['oferta_vl'] = df_vendas.apply(
    lambda row: row['valor'] - 12 if row['promo'] == 'mensal' else
                row['valor'] * 0.7 if row['promo'] == '30_P' else
                row['valor'] * 0.5 if row['promo'] == '50_P' else
                row['valor'],
    axis=1
)

# Exibindo o DataFrame para verificar
df_vendas

# data |	produto	| oferta	| valor	| promo	| vigencia	| oferta_vl	| vigencia_ctt


,data,dia_semana,produto,oferta,valor,promo,vigencia,oferta_vl
0,2024-01-01,seg,C,S,109.9,50_P,3,54.95
1,2024-01-01,seg,D,N,99.9,Sem_Oferta,0,99.90
2,2024-01-01,seg,B,N,119.9,Sem_Oferta,0,119.90
3,2024-01-01,seg,A,N,129.9,Sem_Oferta,0,129.90
4,2024-01-01,seg,D,N,99.9,Sem_Oferta,0,99.90
...,...,...,...,...,...,...,...,...
17950,2025-04-30,qua,C,S,109.9,30_P,3,76.93
17951,2025-04-30,qua,D,N,99.9,Sem_Oferta,0,99.90
17952,2025-04-30,qua,C,N,109.9,Sem_Oferta,0,109.90
17953,2025-04-30,qua,B,S,119.9,50_P,3,59.95


In [ ]:
# Criando o Ticket Médio
# Calculando o ticket médio com base na vigência e no valor da oferta

df_vendas['ticket_medio'] = df_vendas.apply(
    lambda row: round((3 * row['oferta_vl'] + 9 * row['valor']) / 12, 2) if row['vigencia'] == 3 else
                round(row['oferta_vl'], 2),
    axis=1
)

In [50]:
df_vendas

,data,dia_semana,produto,oferta,valor,promo,vigencia,oferta_vl,ticket_medio
0,2024-01-01,seg,C,S,109.9,50_P,3,54.95,96.16
1,2024-01-01,seg,D,N,99.9,Sem_Oferta,0,99.90,99.90
2,2024-01-01,seg,B,N,119.9,Sem_Oferta,0,119.90,119.90
3,2024-01-01,seg,A,N,129.9,Sem_Oferta,0,129.90,129.90
4,2024-01-01,seg,D,N,99.9,Sem_Oferta,0,99.90,99.90
...,...,...,...,...,...,...,...,...,...
17950,2025-04-30,qua,C,S,109.9,30_P,3,76.93,101.66
17951,2025-04-30,qua,D,N,99.9,Sem_Oferta,0,99.90,99.90
17952,2025-04-30,qua,C,N,109.9,Sem_Oferta,0,109.90,109.90
17953,2025-04-30,qua,B,S,119.9,50_P,3,59.95,104.91
